In [31]:
from netCDF4 import Dataset
%matplotlib inline
from pylab import *
from mpl_toolkits.basemap import Basemap

# This function is used to find the nearest point on the grid to the AERI location
def find_index_of_nearest_xy(y_array, x_array, y_point, x_point):
    distance = (y_array-y_point)**2 + (x_array-x_point)**2
    idy,idx = np.where(distance==distance.min())
    return idy[0],idx[0]
# This is used to convert between the relative humidity grid that is stored in the
# model grids to mixing ratio (which is what we need for the prior)
def rh2q(temp, pres, rh):
    Rv = 461.
    L = 2.453 * 10**6
    es = 6.11 * np.exp((L/Rv)*((1./(273.15)) - (1./temp)))
    e = rh * es
    q = (0.622*e) / ((pres/100.) - e)
    return q


def getMotherlodeProfiles(yyyymmddhh='2016012717', time_window=1, aeri_lat=35.45, aeri_lon=-97.62, size=1):
    #This function is used to check for recent RAP data on the Motherlode server
    #This is the function that should be called if we need to run AERIoe in real-time.
    #
    # THIS FUNCTION IS WORKING ON 2/24/2015
    # GETS WHEN I WANT REALTIME RAP OBSERVATIONS 

    recent_rap_path = 'http://thredds.ucar.edu/thredds/dodsC/grib/NCEP/RAP/CONUS_13km/RR_CONUS_13km_' + \
        yyyymmddhh[:8] + '_' + yyyymmddhh[8:10] + '00.grib2/GC'
    print recent_rap_path
    try:
        d = Dataset(recent_rap_path)
        print d.variables.keys()
        print "Found 13 km RAP data for this date on the Motherlode UCAR server."
        type = "RAP13km"
        path = recent_rap_path
    except:
        print "No RAP data found for this date on the Motherlode UCAR server."
        return None
    
    ll = Dataset('13km_latlon.nc')
    lon = ll.variables['lon'][:]
    lat = ll.variables['lat'][:]
    aeri_lon = float(aeri_lon)
    aeri_lat = float(aeri_lat)
    idy, idx = find_index_of_nearest_xy(lon, lat, aeri_lon, aeri_lat)
    
    #print lon, lat
    #print lon.shape, d.variables['Temperature_isobaric'].shape
    #print idy, idx
    #print lon[idy, idx]
    #print lat[idy, idx]
    #print ll.variables
    
    #plt.figure(figsize=(11,8))
    #m = Basemap(llcrnrlon=-120,llcrnrlat=20,urcrnrlat=45, urcrnrlon=-58,
    #                resolution='h',projection='stere',\
    #                lat_ts=50,lat_0=50,lon_0=-97., area_thresh=10000)
    #m.drawcoastlines(color='#999999')
    #m.drawcountries(color='#999999')
    #m.drawstates(color='#999999')
    
    #x_grid, y_grid = m(lon, lat)
    #m.contourf(x_grid, y_grid, ll.variables['Geopotential_height_surface'][:].T, 40)
    #x_pt, y_pt = m(lon[idy, idx], lat[idy, idx])
    #plot(x_pt, y_pt, 'mo')
    #print ll.variables['Geopotential_height_surface'][idy, idx]
    #print ll.variables['Geopotential_height_surface'][idx, idy]
    #colorbar()
    #show()
    #stop
    size = int(size)
    print d.variables['reftime'][:]
    print d.variables['time'][:]
    stop
    pres = d.variables['isobaric'][:] #Pascals
    temp =  d.variables['Temperature_isobaric'][:time_window,:,idy-size:idy+size,idx-size:idx+size] # 
    center_temp =  d.variables['Temperature_isobaric'][:time_window,:,idy, idx] # 
    rh = d.variables['Relative_humidity_isobaric'][:time_window,:,idy-size:idy+size,idx-size:idx+size]
    center_rh = d.variables['Relative_humidity_isobaric'][:time_window,:,idy, idx]
    hght = d.variables['Geopotential_height_isobaric'][:time_window,:,idy-size:idy+size,idx-size:idx+size]
    center_hght = d.variables['Geopotential_height_isobaric'][:time_window,:,idy ,idx]
    sfc_pres = d.variables['Pressure_surface'][:time_window, idy-size:idy+size,idx-size:idx+size]  #Pascals
    center_sfc_pres = d.variables['Pressure_surface'][:time_window, idy, idx]  #Pascals
    sfc_hght = ll.variables['Geopotential_height_surface'][idy-size:idy+size, idx-size:idx+size]
    center_sfc_hght = ll.variables['Geopotential_height_surface'][idx, idy]
    #sfc_hght = d.variables['Geopotential_height_surface'][:time_window, idy-size:idy+size,idx-size:idx+size]
    sfc_temp = d.variables['Temperature_height_above_ground'][:time_window,0, idy-size:idy+size,idx-size:idx+size]
    center_sfc_temp = d.variables['Temperature_height_above_ground'][:time_window,0, idy, idx]
    sfc_rh = d.variables['Relative_humidity_height_above_ground'][:time_window,0, idy-size:idy+size,idx-size:idx+size]
    center_sfc_rh = d.variables['Relative_humidity_height_above_ground'][:time_window,0, idy, idx]
    
    print "i, TEMP, RH, HGHT, PRES"
    for i in range(len(center_temp[0])):
        print i, center_temp[0][i], center_rh[0][i], center_hght[0][i], pres[i]
    print center_sfc_pres[0], center_sfc_temp[0], center_sfc_hght, center_sfc_rh[0]
    
    hght_prof = center_hght[0, :]
    idx_aboveground = np.where(pres < center_sfc_pres[0])[0]
    print idx_aboveground
    new_hght = np.hstack((center_sfc_hght+2, center_hght[0, idx_aboveground][::-1]))
    new_temp = np.hstack((center_sfc_temp[0], center_temp[0, idx_aboveground][::-1]))
    new_rh = np.hstack((center_sfc_rh[0], center_rh[0, idx_aboveground][::-1]))
    new_pres = np.hstack((center_sfc_pres[0], pres[idx_aboveground][::-1]))
    new_q = rh2q(new_temp, new_pres, new_rh/100.)*1000.
    
    print "i, TEMP, RH, HGHT, PRES, WVMR"
    for i in range(len(new_q)):
        print i, new_temp[i], new_rh[i], new_hght[i], new_pres[i], new_q[i]
    plot(new_temp, new_hght)
    show()
    stop
    ll.close()
    d.close()
    
    mxrs = []
    temps = []
    press = []
    for t in range(len(temp)):
        for index, x in np.ndenumerate(sfc_hght):
            hght_prof = hght[t, :, index[0], index[1]]
            idx_aboveground = np.where(sfc_hght[index] < hght_prof)[0]
            new_hght = (np.hstack((sfc_hght[index]+2, hght[t, idx_aboveground, index[0], index[1]][::-1])) - sfc_hght[index])/1000.
            new_temp = np.hstack((sfc_temp[t, index[0], index[1]], temp[t, idx_aboveground, index[0], index[1]][::-1]))
            new_rh = np.hstack((sfc_rh[t, index[0], index[1]], rh[t, idx_aboveground, index[0], index[1]][::-1]))
            new_pres = np.hstack((sfc_pres[t, index[0], index[1]], pres[idx_aboveground][::-1]))
            new_q = rh2q(new_temp, new_pres, new_rh/100.)*1000.
            print new_q, new_temp, new_hght, new_pres 
            print new_q.shape, new_temp.shape, new_hght.shape, new_pres.shape

            stop 
            oe_mxr = np.interp(aerioe_hght, new_hght, new_q)
            oe_temp = np.interp(aerioe_hght, new_hght, new_temp)
            oe_pres = np.interp(aerioe_hght, new_hght, new_pres)
            mxrs.append(oe_mxr)
            temps.append(oe_temp)
            press.append(new_pres)
    
    temps = np.asarray(temps)
    mxrs = np.asarray(mxrs)
    
    return temps, mxrs, press, type, path

In [32]:
getMotherlodeProfiles()

http://thredds.ucar.edu/thredds/dodsC/grib/NCEP/RAP/CONUS_13km/RR_CONUS_13km_20160127_1700.grib2/GC
[u'x', u'y', u'reftime', u'time', u'time1', u'time2', u'pressure_difference_layer', u'height_above_ground_layer', u'height_above_ground_layer1', u'isobaric', u'height_above_ground', u'pressure_difference_layer1', u'height_above_ground1', u'pressure_difference_layer2', u'isobaric_layer', u'LambertConformal_Projection', u'time_bounds', u'time1_bounds', u'pressure_difference_layer_bounds', u'height_above_ground_layer_bounds', u'height_above_ground_layer1_bounds', u'pressure_difference_layer1_bounds', u'pressure_difference_layer2_bounds', u'isobaric_layer_bounds', u'Convective_available_potential_energy_surface', u'Convective_available_potential_energy_pressure_difference_layer', u'Convective_inhibition_surface', u'Convective_inhibition_pressure_difference_layer', u'Convective_precipitation_surface_1_Hour_Accumulation', u'Geopotential_height_cloud_base', u'Geopotential_height_cloud_tops', u'

NameError: global name 'stop' is not defined